## Jakob Ohmayer (4742300)

In [1]:
import os
import zipfile
import torch

import requests
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torchvision.datasets import CIFAR10
from tqdm import tqdm
import torch.nn as nn

### Load model ResNet34

In [2]:
import adapt

In [3]:
from models.resnet import resnet34

In [4]:
threads = 40
torch.set_num_threads(threads)

#maybe better performance
%env OMP_PLACES=cores
%env OMP_PROC_BIND=close
%env OMP_WAIT_POLICY=active

env: OMP_PLACES=cores
env: OMP_PROC_BIND=close
env: OMP_WAIT_POLICY=active


### Choose approximate multiplier 

Here the multiplier SPR12_44 is used.

In [5]:
axx_mult = 'SPR12_44'

### Load model for evaluation

In [6]:
model = resnet34(pretrained=True, axx_mult = axx_mult)

model.eval() # for evaluation

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_SPR12_44/build.ninja...
Building extension module PyInit_conv2d_SPR12_44...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_SPR12_44...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_SPR12_44, skipping build step...
Loading extension module PyInit_conv2d_SPR12_44...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_SPR12_44, skipping build step...
Loading extension module PyInit_conv2d_SPR12_44...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_SPR12_44, skipping bui

ResNet(
  (conv1): AdaPT_Conv2d(
    3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
    (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): AdaPT_Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
        (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(

### Load dataset


In [7]:
def val_dataloader(mean = (0.4914, 0.4822, 0.4465), std = (0.2471, 0.2435, 0.2616)):

    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean, std),
        ]
    )
    dataset = CIFAR10(root="datasets/cifar10_data", train=False, download=True, transform=transform)
    dataloader = DataLoader(
        dataset,
        batch_size=128,
        num_workers=0,
        drop_last=True,
        pin_memory=False,
    )
    return dataloader

transform = T.Compose(
        [
            T.RandomCrop(32, padding=4),
            T.RandomHorizontalFlip(),
            T.ToTensor(),
            T.Normalize(mean = (0.4914, 0.4822, 0.4465), std = (0.2471, 0.2435, 0.2616)),
        ]
    )
dataset = CIFAR10(root="datasets/cifar10_data", train=True, download=True, transform=transform)

evens = list(range(0, len(dataset), 10))
trainset_1 = torch.utils.data.Subset(dataset, evens)

data = val_dataloader()

# data_t is used for calibration purposes and is a subset of train-set
data_t = DataLoader(trainset_1, batch_size=128,
                                            shuffle=False, num_workers=0)


Files already downloaded and verified
Files already downloaded and verified


### Run model calibration for quantization

Calibrates the quantization parameters. Is rerun after each training epoch.

In [ ]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib

def collect_stats(model, data_loader, num_batches):
     """Feed data to the network and collect statistic"""

     # Enable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.disable_quant()
                 module.enable_calib()
             else:
                 module.disable()

     for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
         model(image.cpu())
         if i >= num_batches:
             break

     # Disable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.enable_quant()
                 module.disable_calib()
             else:
                 module.enable()

def compute_amax(model, **kwargs):
 # Load calib result
 for name, module in model.named_modules():
     if isinstance(module, quant_nn.TensorQuantizer):
         if module._calibrator is not None:
             if isinstance(module._calibrator, calib.MaxCalibrator):
                 module.load_calib_amax()
             else:
                 module.load_calib_amax(**kwargs)
         print(F"{name:40}: {module}")
 model.cpu()

def calibrate_model(model, data_t):
    # It is a bit slow since we collect histograms on CPU
    with torch.no_grad():
        stats = collect_stats(model, data_t, num_batches=2)
        amax = compute_amax(model, method="percentile", percentile=99.99)
    
        # optional - test different calibration methods
        #amax = compute_amax(model, method="mse")
        #amax = compute_amax(model, method="entropy")


In [ ]:
calibrate_model(model, data_t)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:17<00:00,  8.85s/it]
W1117 18:53:02.591396 138092802979648 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 18:53:02.592206 138092802979648 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 18:53:02.592847 138092802979648 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 18:53:02.593490 138092802979648 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 18:53:02.594076 138092802979648 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 18:53:02.594689 138092802979648 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 18:53:02.595309 138092802979648 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 18:53:02.595905 138092802979648 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 18:53:02.596551 138092802979648 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 18:53:02.

W1117 18:53:02.638047 138092802979648 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 18:53:02.638700 138092802979648 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 18:53:02.639321 138092802979648 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 18:53:02.639934 138092802979648 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 18:53:02.640563 138092802979648 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 18:53:02.641210 138092802979648 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 18:53:02.641852 138092802979648 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 18:53:02.642493 138092802979648 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 18:53:02.643128 138092802979648 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 18:53:02.643766 138092802979648

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1418 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6620 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0555 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.3258 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0327 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6022 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0395 calibrator=HistogramCalibrator quant)
layer1.1

### Run model evaluation (before re-training)

In [11]:
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [29:07<00:00, 22.41s/it]

1747.6586084780001
Accuracy of the network on the 10000 test images: 33.7740 %


Accuracy before re-training: 33.77%

### Run approximate-aware re-training for 15 epochs


In [ ]:
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

EPOCHS = 15

# finetune the model for one epoch based on data_t subset
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", 0, 1)
    calibrate_model(model)

### Rerun model evaluation after re-training

In [ ]:
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Accuracy after re-training: